# Crystal plasticity formulations

XXX

## Importing required libraries and setting up

This example will run on the GPU with CUDA if available.  `nchunk` is the chunk size for the `pyzag` integration, `ncrystal` is the number of random initial crystal orientations to use.

This example simulates rolling in an FCC material.  `rate` sets the strain rate, `total_rolling_strain` the reduction strain, and `ntime` the number of time steps to take.

We need to calculate both the deformation rate/vorticity and the deformation gradient

In [1]:
import sys
sys.path.append("/home/messner/packages/pyzag")
import matplotlib.pyplot as plt
import torch
import neml2
import neml2.tensors
from pyzag import nonlinear, chunktime
import neml2.postprocessing

In [2]:
torch.set_default_dtype(torch.double)
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

nchunk = 10
ncrystal = 500

In [3]:
rate = 0.0001
total_rolling_strain = 0.5
ntime = 2500
end_time = total_rolling_strain / rate
initial_orientations = neml2.tensors.Rot.fill_random(ncrystal).torch().to(device)

In [4]:
deformation_rate = torch.zeros((ntime, ncrystal, 6), device = device)
deformation_rate[:, :, 1] = rate
deformation_rate[:, :, 2] = -rate
times = torch.linspace(0, end_time, ntime, device = device).unsqueeze(-1).unsqueeze(-1).expand((ntime, ncrystal, 1))
vorticity = torch.zeros((ntime, ncrystal, 3), device = device)

In [5]:
# Now get the integrated deformation gradient
full_deformation_rate = neml2.tensors.R2(neml2.tensors.SR2(deformation_rate)).torch()
F = torch.zeros_like(full_deformation_rate)
F[0] = torch.eye(3, device = device)
dt = torch.diff(times, dim = 0)
Finc = torch.linalg.matrix_exp(full_deformation_rate[:-1] * dt.unsqueeze(-1))
for i in range(1, ntime):
    F[i] = Finc[i-1] @ F[i - 1]

## pyzag driver for the separate model

For more details on the pyzag driver used here see the deterministic and stochastic inference examples.  This helper object just lets us integrate the crystal model using pyzag

In [6]:

class SolveSeparate(torch.nn.Module):
    """Just integrate the model through some strain history

    Args:
        discrete_equations: the pyzag wrapped model
        nchunk (int): number of vectorized time steps
        rtol (float): relative tolerance to use for Newton's method during time integration
        atol (float): absolute tolerance to use for Newton's method during time integration
    """
    def __init__(self, discrete_equations, nchunk = 1, rtol = 1.0e-6, atol = 1.0e-8):
        super().__init__()
        self.discrete_equations = discrete_equations
        self.nchunk = nchunk
        self.rtol = rtol
        self.atol = atol

    def forward(self, time, deformation_rate, vorticity, initial_orientations = None):
        """Integrate through some time/temperature/strain history and return stress
        Args:
            time (torch.tensor): batched times
            deformation_rate (torch.tensor): batched deformation rates
            vorticity (torch.tensor): batched vocticities

        Keyword Args:
            initial_orientation (torch.tensor): if provided, the initial orientations for each crystal
        """
        solver = nonlinear.RecursiveNonlinearEquationSolver(
            self.discrete_equations, step_generator = nonlinear.StepGenerator(self.nchunk),
            predictor=nonlinear.PreviousStepsPredictor(), nonlinear_solver = chunktime.ChunkNewtonRaphson(rtol = self.rtol, atol = self.atol)
        )


        # Setup
        forces = self.discrete_equations.forces_asm.assemble_by_variable({'forces/t': time, 'forces/deformation_rate': deformation_rate, 'forces/vorticity': vorticity}).torch()
        state0 = self.discrete_equations.state_asm.split_by_variable(neml2.Tensor(torch.zeros(forces.shape[1:-1] + (self.discrete_equations.nstate,), device = forces.device), 1))
        if initial_orientations is not None:
            state0['state/orientation'] = neml2.Tensor(initial_orientations, 1)
        state0 = self.discrete_equations.state_asm.assemble_by_variable(state0).torch()

        result =  nonlinear.solve_adjoint(
            solver, state0, len(forces), forces
        )

        return result

## Simulate rolling deformation for the seperated model

Simulate the rolling deformation and extract the final crystal orientations

In [7]:
nmodel_separate = neml2.load_model("crystal.i", "implicit_rate")
nmodel_separate.to(device = device)
model_separate = SolveSeparate(neml2.pyzag.NEML2PyzagModel(nmodel_separate), nchunk = nchunk)

In [8]:
with torch.no_grad():
    orientations_separate = neml2.tensors.Rot(model_separate(times, deformation_rate, vorticity, initial_orientations = initial_orientations)[-1,:,6:9])

## ODF reconstruction for the seperate model

Reconstruct the ODF from the discrete data.  This example optimizes the kernel half-width with the build in routine (which uses a cross-validation approach).  Print the final, optimal half width.

In [9]:
odf_separate = neml2.postprocessing.odf.KDEODF(orientations_separate, neml2.postprocessing.odf.DeLaValleePoussinKernel(torch.tensor(0.1)))
odf_separate.optimize_kernel(verbose = True)
print(odf_separate.kernel.h)

loss: -3.43226e+00: 100%|██████████| 50/50 [00:29<00:00,  1.71it/s]

Parameter containing:
tensor(0.0476, requires_grad=True)


## pyzag driver for the multiplicative model

Same thing as for the separated model, just now for the multiplicative form where we take the deformation gradient as input.  Don't forget to set the initial plastic deformation gradient to the identity!

In [10]:

class SolveMultiplicative(torch.nn.Module):
    """Just integrate the model through some strain history

    Args:
        discrete_equations: the pyzag wrapped model
        nchunk (int): number of vectorized time steps
        rtol (float): relative tolerance to use for Newton's method during time integration
        atol (float): absolute tolerance to use for Newton's method during time integration
    """
    def __init__(self, discrete_equations, nchunk = 1, rtol = 1.0e-6, atol = 1.0e-8):
        super().__init__()
        self.discrete_equations = discrete_equations
        self.nchunk = nchunk
        self.rtol = rtol
        self.atol = atol

    def forward(self, time, deformation_gradient, initial_orientations = None):
        """Integrate through some time/temperature/strain history and return stress
        Args:
            time (torch.tensor): batched times
            deformation_gradient (torch.tensor): batched deformation gradients

        Keyword Args:
            initial_orientation (torch.tensor): if provided, the initial orientations for each crystal
        """
        solver = nonlinear.RecursiveNonlinearEquationSolver(
            self.discrete_equations, step_generator = nonlinear.StepGenerator(self.nchunk),
            predictor=nonlinear.PreviousStepsPredictor(), nonlinear_solver = chunktime.ChunkNewtonRaphson(rtol = self.rtol, atol = self.atol)
        )

        # Setup
        forces = self.discrete_equations.forces_asm.assemble_by_variable({'forces/t': time, 'forces/F': deformation_gradient.flatten(start_dim=-2), 'forces/r': initial_orientations.unsqueeze(0).expand(time.shape[:-1]+(3,))}).torch()
        state0 = self.discrete_equations.state_asm.split_by_variable(neml2.Tensor(torch.zeros(forces.shape[1:-1] + (self.discrete_equations.nstate,), device = forces.device), 1))
        state0['state/Fp'] = neml2.tensors.Tensor(torch.eye(3, device = time.device).unsqueeze(0).expand(time.shape[1:2]+(3,3)), 1)
        state0 = self.discrete_equations.state_asm.assemble_by_variable(state0).torch()

        result =  nonlinear.solve_adjoint(
            solver, state0, len(forces), forces
        )

        return result

## Simulate for the multiplicative model
Again, simulate rolling deformation

In [11]:
nmodel_mult = neml2.load_model("crystal_multiplicative.i", "implicit_rate")
nmodel_mult.to(device = device)
model_mult = SolveMultiplicative(neml2.pyzag.NEML2PyzagModel(nmodel_mult), nchunk = nchunk)

In [12]:
with torch.no_grad():
    results_mult = model_mult(times, F, initial_orientations = initial_orientations)[-1]

## Extract the orientations for the multiplicative model

This takes some doing.  We first need to get $F_p$ from the state, then calculate $F_e = F F_p^{-1}$, then do a polar decomposition to get the rotation as a matrix, then convert to modified Rodrigues parameters.

In [13]:
split_state = model_mult.discrete_equations.state_asm.split_by_variable(neml2.Tensor(results_mult,0))
Fp = split_state["state/Fp"].torch().reshape(-1,3,3)
Flast = F[-1]
Fe = Flast @ torch.linalg.inv(Fp)
U, S, Vh = torch.linalg.svd(Fe)


KeyError: 'state/Fp'

## Plot the pole figures

Use the reconstructed ODFs to plot continuous 111 polefigure

In [ ]:
neml2.postprocessing.pretty_plot_pole_figure_odf(odf_separate, torch.tensor([1,1,1.0], device = device), crystal_symmetry = "432", limits = (0.0, 3.0), ncontour = 12)